In [1]:
import numpy as np
import pandas as pd
import string
from gensim.models.word2vec import Word2Vec
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Conv1D, MaxPool1D, GlobalMaxPool1D, Embedding, Activation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from sklearn import preprocessing

In [2]:
data = pd.read_csv("dataset.csv")

In [3]:
from sklearn.model_selection import train_test_split

x = data['text']
y = data['classes']

X_train,X_test,y_train,y_test=train_test_split(x,y,random_state=1,test_size=0.2,shuffle=True)

In [4]:
# Preprocess the text data
stop_words = set(stopwords.words('english'))
def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

X_train = X_train.apply(preprocess)
X_test = X_test.apply(preprocess)

In [5]:
# Train the Word2Vec model
sentences = [sentence.split() for sentence in X_train]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)

In [6]:
token = Tokenizer(7229)
token.fit_on_texts(data['text'])
text = token.texts_to_sequences(data['text'])
text = pad_sequences(text, 75)

In [7]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(data['classes'])
y = to_categorical(y)

In [8]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(np.array(text),y,random_state=1,test_size=0.2,shuffle=True)

In [9]:
def gensim_to_keras_embedding(model, train_embeddings=False):
    """Get a Keras 'Embedding' layer with weights set from Word2Vec model's learned word embeddings.

    Parameters
    ----------
    train_embeddings : bool
        If False, the returned weights are frozen and stopped from being updated.
        If True, the weights can / will be further updated in Keras.

    Returns
    -------
    `keras.layers.Embedding`
        Embedding layer, to be used as input to deeper network layers.

    """
    keyed_vectors = model.wv  # structure holding the result of training
    weights = keyed_vectors.vectors  # vectors themselves, a 2D numpy array    
    index_to_key = keyed_vectors.index_to_key  # which row in `weights` corresponds to which word?

    layer = Embedding(
        input_dim=weights.shape[0],
        output_dim=weights.shape[1],
        weights=[weights],
        trainable=train_embeddings,
    )
    return layer

In [20]:
keras_model1 = Sequential()
def build_model(model):
    model.add(gensim_to_keras_embedding(w2v_model))
    model.add(Dropout(0.2))
    model.add(Conv1D(300, 3, activation='relu', padding='same', strides=1))
    model.add(Dropout(0.2))
    model.add(GlobalMaxPool1D())
    model.add(Dense(200))
    model.add(Dropout(0.2))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    model.compile(loss='binary_crossentropy', metrics=['acc'], optimizer='adam')

build_model(keras_model1)
keras_model1.fit(X_train, y_train, batch_size=16, epochs=30, validation_data=(X_test, y_test))

Epoch 1/30
3200/3200 [==============================] - 17s 5ms/step - loss: 0.4363 - acc: 0.8070 - val_loss: 0.2758 - val_acc: 0.8691
Epoch 2/30
3200/3200 [==============================] - 15s 5ms/step - loss: 0.2958 - acc: 0.8617 - val_loss: 0.2552 - val_acc: 0.8803
Epoch 3/30
3200/3200 [==============================] - 15s 5ms/step - loss: 0.2774 - acc: 0.8729 - val_loss: 0.2465 - val_acc: 0.8838
Epoch 4/30
3200/3200 [==============================] - 15s 5ms/step - loss: 0.2608 - acc: 0.8823 - val_loss: 0.2441 - val_acc: 0.8898
Epoch 5/30
3200/3200 [==============================] - 16s 5ms/step - loss: 0.2524 - acc: 0.8864 - val_loss: 0.2789 - val_acc: 0.8752
Epoch 6/30
3200/3200 [==============================] - 16s 5ms/step - loss: 0.2450 - acc: 0.8913 - val_loss: 0.2888 - val_acc: 0.8798
Epoch 7/30
3200/3200 [==============================] - 16s 5ms/step - loss: 0.2383 - acc: 0.8969 - val_loss: 0.2575 - val_acc: 0.8870
Epoch 8/30
3200/3200 [==============================] -

In [21]:
from sklearn.metrics import classification_report

y_predict = keras_model1.predict(X_test)
predicted_categories = np.argmax(y_predict, axis=1)

true_categories_argmax = np.argmax(y_test, axis=1)

print(classification_report(true_categories_argmax, predicted_categories))

400/400 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      6528
           1       0.89      0.89      0.89      6272

    accuracy                           0.89     12800
   macro avg       0.89      0.89      0.89     12800
weighted avg       0.89      0.89      0.89     12800

